# Eigenmode simulation: floating transmon
Eigenmode simulation and EPR analysis of a floating transmon qubit coupled to a $\lambda/2$ readout resonator in hanger-mode with a $50\Omega$-terminated transmission line segment.
- Creates Qiskit Metal design
- Exports design to `.gds` file
- Creates GMSH mesh file (`.msh`) with ports

In [ ]:
import qiskit_metal as metal
from qiskit_metal import MetalGUI, Dict
from qiskit_metal.qlibrary.tlines.meandered import RouteMeander
from qiskit_metal.qlibrary.couplers.coupled_line_tee import CoupledLineTee
from qiskit_metal.qlibrary.terminations.open_to_ground import OpenToGround
from qiskit_metal.analyses import cpw_calculations
from SQDMetal.Utilities.Materials import Material
from SQDMetal.Utilities.QUtilities import QUtilities
from SQDMetal.Utilities.CpwParams import CpwParams
from SQDMetal.Utilities.QubitDesigner import ResonatorHalfWave
from qiskit_metal.qlibrary.tlines.straight_path import RouteStraight
from SQDMetal.Utilities.MakeGDS import MakeGDS
from importlib import reload
import warnings
import contextlib
import sys
import os
from pathlib import Path
%matplotlib inline
warnings.filterwarnings("ignore")

# create directories for saving if not present
gds_dir = Path("gds")
gds_dir.mkdir(exist_ok=True)

In [ ]:
### DESIGN NAME (UPDATE)
name = "example_floatingTransmon"

### Qubit, resonator frequencies
qubit_freq_GHz = 4.5
res_freq_GHZ = 6.8

## Design

In [ ]:
# Create a DesignPlanar (class to handle 2D designs)
design = metal.designs.DesignPlanar(metadata=Dict(design_name=name))
design.overwrite_enabled = True  # Enables overwriting components
design.chips.main.size.size_x = "4mm"
design.chips.main.size.size_y = "4mm"
design.chips.main.size.size_z = "500um"
design.chips.main.size.center_x = "0mm"
design.chips.main.size.center_y = "0mm"

# a_gds = design.renderers.gds
# a_gds.options["path_filename"] = (
#     "C:\Jupyter_notebooks\chip_design_base_env/FakeJunction_01.GDS"
# )

# Resonator parameters
resonator = ResonatorHalfWave(res_freq_GHZ * 1e9)
m = Material("siliconcryo")

# CPW parameters
cpw_width = 10e-6
cpw = CpwParams(m.permittivity, 500e-6)
cpw_gap = cpw.get_gap_from_width(cpw_width)

l_fullwave, _, _ = cpw_calculations.guided_wavelength(
                freq=resonator.f0,
                line_width=cpw_width,
                line_gap=cpw_gap,
                substrate_thickness=cpw.dielectric_thickness,
                film_thickness=200e-9,
            )
l_halfwave = l_fullwave / 2

Resonator length (f = 6.8 GHz): 8.96 mm


## Draw components

In [ ]:
# Import the modules and reload
import SQDMetal.Comps.Qubits as Qubits
TransmonTaperedInsets = Qubits.TransmonTaperedInsets

# tapered transmon
Q1 = TransmonTaperedInsets(design, 
                           "Q1",
                           options=dict(
                                pos_x="0.5mm",
                                pos_y="1mm",
                                orientation='90',
                                connection_pads=dict(
                                    readout=dict(
                                           pad_gap="100um", 
                                        )
                                    ),
                                inductor_height='100um',
                                pad_gap='100um',
                                taper_height='0um',
                                )  
                            )

##Coupling region
q1_read_T = CoupledLineTee(
    design,
    "Q1_Read_T",
    options=dict(
        pos_x="-1mm",
        pos_y="0mm",
        orientation="180",
        coupling_space="5um",
        coupling_length="220um",
        down_length="100um",
        prime_width=f"{cpw_width*1e6}um",
        prime_gap=f"{cpw_gap*1e6}um",
        second_width=f"{cpw_width*1e6}um",
        second_gap=f"{cpw_gap*1e6}um",
        open_termination=True,
    ),
)

##Feedline
# Left open
otg_left = OpenToGround(
    design,
    "otg_left",
    options=dict(
        pos_x='-1.8mm',
        pos_y='0um',
        orientation='180',
        width=f"{cpw_width*1e6}um",
        gap=f"{cpw_gap*1e6}um",
    )
)
# Right open
otg_right = OpenToGround(
    design,
    "otg_right",
    options=dict(
        pos_x='0mm',
        pos_y='0um',
        orientation='0',
        width=f"{cpw_width*1e6}um",
        gap=f"{cpw_gap*1e6}um",
    )
)
feedline = RouteStraight(
    design,
    "feedline_main",
    options=dict(
        pin_inputs=dict(
            start_pin=dict(component='otg_left', pin='open'),
            end_pin=dict(component='otg_right', pin='open'),
        ),
        trace_width=f"{cpw_width*1e6}um",
        trace_gap=f"{cpw_gap*1e6}um",
    ),
)

##Readout resonator
#calculate extra lengths on resonator from coupling regions
res_coupler_l_mm = QUtilities.calc_points_on_path([0], design, component_name="Q1_Read_T", trace_name='prime_cpw_sub')[-1]
q_coupler_l_mm = QUtilities.calc_points_on_path([0], design, component_name="Q1", trace_name='readout_wire')[-1]
extra_length_mm = res_coupler_l_mm + q_coupler_l_mm
read_q1 = RouteMeander(
    design,
    "Read_Q1",
    options=dict(
        hfss_wire_bonds=True,
        pin_inputs=Dict(
            start_pin=Dict(component="Q1", pin="readout"),
            end_pin=Dict(component="Q1_Read_T", pin="second_end"),
        ),
        lead=Dict(
            start_straight="125um",
            end_straight="125um",
        ),
        meander=Dict(
            asymmetry="0um",
            spacing="100um",
        ),
        trace_width=f"{cpw_width*1e6}um",
        trace_gap=f"{cpw_gap*1e6}um",
        fillet="40um",
        total_length=f"{(l_halfwave * 1e3) - extra_length_mm:.2f}mm",
    ),
)

##GDS export
design.rebuild()
export_name = design.get_design_name() + ".gds"
gds_out = MakeGDS(design)
gds_out.export("gds/" + export_name, export_type="positive")



GDS exported at c:\Users\Experiment\Desktop\ZachD\Repos\NQCT-Design\NQCT-01-01\Q1_4.5GHz\gds\NQCT-01-01_Q1.gds


## Capacitance simulation

In [ ]:
import SQDMetal.PALACE.Capacitance_Simulation as CapSim
from SQDMetal.PALACE.Capacitance_Simulation import PALACE_Capacitance_Simulation
            
############################################################

# Create the Palace Eigenmode simulation
sim_name = design.get_design_name()

# Palace simulation options
user_defined_options = {
    "mesh_refinement":  0,                             #refines mesh in PALACE - essetially divides every mesh element in half
    "dielectric_material": "silicon",                  #choose dielectric material - 'silicon' or 'sapphire'
    "starting_freq": 3e9,                              #starting frequency in Hz 
    "number_of_freqs": 3,                              #number of eigenmodes to find
    "solns_to_save": 3,                                #number of electromagnetic field visualizations to save
    "solver_order": 2,                                 #increasing solver order increases accuracy of simulation, but significantly increases sim time
    "solver_tol": 1.0e-8,                              #error residual tolerance foriterative solver
    "solver_maxits": 200,                              #number of solver iterations
    "mesh_sampling": 130,                              #number of points to mesh along a geometry
    "fillet_resolution":12,                            #number of vertices per quarter turn on a filleted path
    'palace_mode': 'wsl',
    'palace_wsl_spack_repo_directory': '~/repo',
    'palace_dir': '~/repo/spack/opt/spack/linux-zen2/palace-develop-5zmriht6rd6ijchmfjmtl3a7pol3ijfn/bin/palace',   # ADD YOUR PATH TO THE PALACE BINARY
    'num_cpus': 16
}

cap_sim = PALACE_Capacitance_Simulation(name = "example_floatingTransmon",            #name of simulation
                                        metal_design = design,                  #feed in qiskit metal design
                                        sim_parent_directory = "",              #choose directory where mesh file, config file and HPC batch file will be saved
                                        mode = 'simPC',                         #choose simulation mode 'HPC' or 'simPC'                                          
                                        meshing = 'GMSH',                       #choose meshing 'GMSH' or 'COMSOL'
                                        user_options = user_defined_options,    #provide options chosen above
                                        view_design_gmsh_gui = False,           #view design in GMSH gui 
                                        create_files = True)  

#Add geometries
cap_sim.add_metallic(1)
cap_sim.add_ground_plane()

#Define meshing
cap_sim.fine_mesh_around_comp_boundaries(
    design.components.keys(),
    mesh_sampling=160,
    min_size=12e-6,
    max_size=250e-6,
    taper_dist_min=12e-6,
    taper_dist_max=250e-6,
    metals_only=True,
)

#Prepare simulation (and mesh)
cap_sim.prepare_simulation()

#Run simulation
capMat = cap_sim.run()

In [ ]:
# cap_sim.display_conductor_indices()

# Test E_C calculation from cap matrix
r = cap_sim.floatingTransmon_calc_params(print_all_capacitances=True, 
                                            res=resonator, 
                                            qubit_freq=(qubit_freq_GHz * 1e9)
                                            )

Capacitance Results
--------------------
C1-Ground        =     79.467 fF
C2-Ground        =     61.430 fF
C1-Readout       =      1.137 fF
C2-Readout       =     10.370 fF
C1-Feedline      =      0.001 fF
C2-Feedline      =      0.004 fF
Mutual C12       =     43.844 fF
C_sigma          =     81.818 fF

Readout Resonator
-------------------
f_res            =      6.800 GHz
L_res            =    745.009 pH
C_res            =    735.294 fF
C_res-Feedline   =      3.318 fF

Circuit Parameters
-------------------
f_qubit          =      4.500 GHz
E_C              =    236.749 MHz
g                =     57.971 MHz
chi              =      0.136 MHz
Delta            =     -2.300 GHz
Anharmonicity    =    236.846 MHz
kappa            =      0.087 MHz
T_1,P            =     18.068 ms

Target Junction Parameters
 (for f_qubit = 4.5 GHz)
--------------------------
E_J              =     11.846 GHz
L_J              =     13.799 nH

E_J/E_C          =     50.037


## Eigenmode simulation

In [ ]:
from SQDMetal.PALACE.Eigenmode_Simulation import PALACE_Eigenmode_Simulation
from SQDMetal.Utilities.Materials import MaterialInterface

eigen_sim = PALACE_Eigenmode_Simulation(
    name="example_floatingTransmon",  # name of simulation
    metal_design=design,  # feed in qiskit metal design
    sim_parent_directory="",  # choose directory where mesh file, config file and HPC batch file will be saved
    mode="simPC",  # choose simulation mode 'HPC' or 'simPC'
    meshing="GMSH",  # choose meshing 'GMSH' or 'COMSOL'
    user_options=user_defined_options,  # provide options chosen above
    view_design_gmsh_gui=False,  # view design in GMSH gui
    create_files=True,
)  # create mesh, config and HPC batch files

# add components and ports
eigen_sim.add_metallic(1)
eigen_sim.add_ground_plane()
eigen_sim.create_port_JosephsonJunction("Q1", L_J=r['L_J_nH'] * 1e-9)
eigen_sim.create_port_CPW_on_Route("feedline_main", "start")
eigen_sim.create_port_CPW_on_Route("feedline_main", "end")

eigen_sim.fine_mesh_around_comp_boundaries(
    design.components.keys(),
    mesh_sampling=160,
    min_size=10e-6,
    max_size=200e-6,
    taper_dist_min=12e-6,
    metals_only=True,
)

# Run post-processing for participation ratios for lossy interfaces
eigen_sim.setup_EPR_interfaces(
    metal_air=MaterialInterface("Aluminium-Vacuum"),
    substrate_air=MaterialInterface("Silicon-Vacuum"),
    substrate_metal=MaterialInterface("Silicon-Aluminium"),
)

# prepare and run simulation
eigen_sim.prepare_simulation()

eigen_sim.run()

wsl: Failed to start the systemd user session for 'experiment'. See journalctl for more details.
==> Error: cannot initialize environment, spack-env/spack.yaml already exists
>> /home/experiment/repo/spack-env/.spack-env/view/bin/mpirun -n 16 /home/experiment/repo/spack/opt/spack/linux-zen2/palace-develop-5zmriht6rd6ijchmfjmtl3a7pol3ijfn/bin/palace-x86_64.bin NQCT-01-01_EPR.json

_____________     _______
_____   __   \____ __   /____ ____________
____   /_/  /  __ ` /  /  __ ` /  ___/  _ \
___   _____/  /_/  /  /  /_/  /  /__/  ___/
/__/     \___,__/__/\___,__/\_____\_____/


--> Warning!
Output folder is not empty; program will overwrite content! (outputFiles)
Git changeset ID: v0.14.0-267-g3ecd1ad0
Running with 16 MPI processes
Device configuration: cpu
Memory configuration: host-std
libCEED backend: /cpu/self/xsmm/blocked

Added 7605 elements in 2 iterations of local bisection for under-resolved interior boundaries
Added 8540 duplicate vertices for interior boundaries in the mesh
A

In [ ]:
e = eigen_sim.calculate_hamiltonian_parameters_EPR(print_output=True)

Simulation Mode Frequencies:
        Freq (GHz)
mode_1    4.252800
mode_2    6.633172
mode_3    7.561018
______________________________

Renormalised Frequencies:
 ***Freqs from simulation are adjusted for Lamb shift
        Freq (GHz)
mode_1    4.069318
mode_2    6.633087
mode_3    7.561018
______________________________

Participation Ratios:
                jj_1
mode_1  9.802960e-01
mode_2  2.900015e-04
mode_3  3.243507e-07
______________________________

Chi Matrix (MHz):
 ***Diag is Anharmonicity, Off-Diag is Cross-Kerr
            mode_1        mode_2        mode_3
mode_1  183.397796  1.692440e-01  2.157680e-04
mode_2    0.169244  3.904564e-05  9.955801e-08
mode_3    0.000216  9.955801e-08  6.346289e-11
______________________________

Lamb Shifts:
        Lamb Shift (MHz)
mode_1        183.482526
mode_2          0.084661
mode_3          0.000108
______________________________

Detuning (GHz):
          mode_1    mode_2    mode_3
mode_1  0.000000 -2.380371 -3.308218
mode_2  2.3803